In [22]:
import requests
import pandas as pd

# No Matchups Yet
# league_id = 781936337963036672

# Redraft Last Place
league_id = 716027744466583552
filename = "Redraft_Matchups.xlsx"

# Old League
league_id = 383720128937734144
filename = "Old_Matchups.xlsx"

# Moved League
league_id = 516329465491447808
filename = "Moved_Matchups.xlsx"

In [23]:
# Fetch NFL Players
URL = "https://api.sleeper.app/v1/players/nfl"
response = requests.get(URL)
player_dictionary = response.json()

# Create lookup dictionaries for roster_id to user_id and roster_id to display_name
URL = f"https://api.sleeper.app/v1/league/{league_id}/rosters"
response = requests.get(URL)
rosters = response.json()
roster_id2user_id = {roster["roster_id"]: roster["owner_id"] for roster in rosters}

URL = f"https://api.sleeper.app/v1/league/{league_id}/users"
response = requests.get(URL)
users = response.json()
user_id2display_name = {user["user_id"]: user["display_name"] for user in users}

roster_id2display_name = {roster["roster_id"]: user_id2display_name[roster["owner_id"]] for roster in rosters}

# Get players used in each matchup and the points they scored

columns = ["week", "matchup_id", "roster_id", "display_name", "player_id", "player", "starter", "points"]
df = []

# Find Max Weeks
week = 1
URL = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}"
response = requests.get(URL)
matchup_rosters = response.json()
while len(matchup_rosters) != 0:
    for roster in matchup_rosters:
        matchup_id = roster["matchup_id"]
        roster_id = roster["roster_id"]
        display_name = roster_id2display_name[roster_id]
        starters = roster['starters']
        for player_id, points in roster["players_points"].items():
            # IKE: USE THIS TO GET PLAYERS TO MATCH UP WITH ROSTERS
            player = player_dictionary[player_id]["first_name"] + " " + player_dictionary[player_id]["last_name"]
            if player_id in starters:
                starter = 'Y'
            else:
                starter = 'N'
            new_row = [week, matchup_id, roster_id, display_name, player_id, player, starter, points]
            df.append(new_row)
    week += 1
    URL = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}"
    response = requests.get(URL)
    matchup_rosters = response.json()

matchup_df = pd.DataFrame(df, columns = columns)
matchup_df.to_excel(f"../Data/{filename}", index = False)
matchup_df

,week,matchup_id,roster_id,display_name,player_id,player,starter,points
0,1,2.0,1,ShoooterMcGavin,SF,San Francisco 49ers,N,6.00
1,1,2.0,1,ShoooterMcGavin,BAL,Baltimore Ravens,Y,18.00
2,1,2.0,1,ShoooterMcGavin,947,Julio Jones,Y,28.70
3,1,2.0,1,ShoooterMcGavin,7085,Isaiah Coulter,N,0.00
4,1,2.0,1,ShoooterMcGavin,7062,Rodrigo Blankenship,Y,7.00
...,...,...,...,...,...,...,...,...
4556,17,NaN,10,evanprinze,289,Drew Brees,Y,20.04
4557,17,NaN,10,evanprinze,2583,Tyrell Williams,N,0.00
4558,17,NaN,10,evanprinze,2161,Jerick McKinnon,N,1.60
4559,17,NaN,10,evanprinze,1817,Sammy Watkins,N,0.00


In [11]:
user_id2display_name

{'331210544826572800': 'ikei',
 '332338685473198080': 'keaton22',
 '332770457449156608': 'jfortkort5',
 '342844660584181760': 'vunosh',
 '342853271066509312': 'sammeroney',
 '342915246849048576': 'adambloom',
 '342922622390083584': 'KingDarsh',
 '342926147304140800': 'johale',
 '453689779289059328': 'Conner123',
 '471379621594525696': 'mullismc',
 '471894725624852480': 'chapelf',
 '600874823534571520': 'Placejt'}

In [8]:
player_dictionary

{'2103': {'fantasy_positions': ['OL'],
  'birth_state': None,
  'hashtag': '#CodyBooth-NFL-FA-60',
  'injury_status': None,
  'player_id': '2103',
  'sportradar_id': '4cd4976e-e230-4935-ad3f-c12876a41350',
  'swish_id': None,
  'fantasy_data_id': 16426,
  'years_exp': 1,
  'age': 27,
  'last_name': 'Booth',
  'depth_chart_order': None,
  'search_full_name': 'codybooth',
  'weight': '285',
  'status': 'Inactive',
  'position': 'OT',
  'injury_body_part': None,
  'number': 60,
  'yahoo_id': 27841,
  'height': '6\'5"',
  'birth_city': None,
  'news_updated': None,
  'rotoworld_id': None,
  'first_name': 'Cody',
  'injury_start_date': None,
  'search_first_name': 'cody',
  'full_name': 'Cody Booth',
  'sport': 'nfl',
  'high_school': None,
  'search_last_name': 'booth',
  'stats_id': None,
  'injury_notes': None,
  'gsis_id': None,
  'birth_date': '1991-04-22',
  'depth_chart_position': None,
  'active': False,
  'birth_country': None,
  'espn_id': 17054,
  'search_rank': 9999999,
  'pract

In [21]:
season = 2021
sport = "nfl"
user_id = 331210544826572800
URL = f"https://api.sleeper.app/v1/user/{user_id}/leagues/{sport}/{season}"
response = requests.get(URL)
leagues = response.json()
leagues

[{'total_rosters': 10,
  'status': 'complete',
  'sport': 'nfl',
  'shard': 888,
  'settings': {'max_keepers': 1,
   'draft_rounds': 5,
   'trade_review_days': 0,
   'squads': 1,
   'reserve_allow_dnr': 1,
   'capacity_override': 0,
   'pick_trading': 1,
   'disable_trades': 0,
   'taxi_years': 2,
   'taxi_allow_vets': 1,
   'best_ball': 0,
   'last_report': 14,
   'disable_adds': 0,
   'waiver_type': 0,
   'bench_lock': 0,
   'reserve_allow_sus': 1,
   'type': 2,
   'reserve_allow_cov': 1,
   'waiver_clear_days': 1,
   'daily_waivers_last_ran': 30,
   'waiver_day_of_week': 2,
   'start_week': 1,
   'playoff_teams': 6,
   'num_teams': 10,
   'reserve_slots': 5,
   'playoff_round_type': 0,
   'daily_waivers_hour': 0,
   'waiver_budget': 100,
   'reserve_allow_out': 1,
   'offseason_adds': 1,
   'last_scored_leg': 17,
   'playoff_seed_type': 0,
   'daily_waivers': 0,
   'divisions': 2,
   'playoff_week_start': 15,
   'daily_waivers_days': 1093,
   'league_average_match': 0,
   'leg': 17,

In [20]:
URL = "https://api.sleeper.app/v1/players/nfl/{season}"
response = requests.get(URL)
player_dictionary = response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)